In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [6]:
def mp_detect(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [7]:
def mp_draw(image, results):
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_hand_landmarks_style())
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_hand_landmarks_style())

In [8]:
MP_POSE_KPS = 33
MP_HAND_KPS = 21

def mp_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() \
    if results.pose_landmarks \
    else np.zeros(MP_POSE_KPS*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() \
    if results.left_hand_landmarks \
    else np.zeros(MP_HAND_KPS*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() \
    if results.right_hand_landmarks \
    else np.zeros(MP_HAND_KPS*3)
    return np.concatenate([pose, lh, rh])

In [41]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset') 

# Actions that we try to detect
actions = np.array(['hello', 'my', 'name', 'I', 'learning', 'sign', 'friend', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 20

# Videos are going to be 30 frames in length
sequence_length = 30

last_file_number = 20
file_nums = range(last_file_number, last_file_number+no_sequences)

In [42]:
for action in actions: 
    for sequence in file_nums:
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [43]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in file_nums:
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mp_detect(frame, holistic)
#                 print(results)

                # Draw landmarks
                mp_draw(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = mp_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(2,no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [30]:
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(X.shape[1],X.shape[-1])))
    model.add(LSTM(128, return_sequences=True, activation='tanh'))
    model.add(LSTM(64, return_sequences=False, activation='tanh'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))
    return model
model = create_lstm_model()

In [31]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(136, 30, 258) (8, 30, 258) (136, 8) (8, 8)


In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 8)                 264       
                                                                 
Total params: 237,416
Trainable params: 237,416
Non-tr

In [34]:
model.fit(X_train, Y_train, epochs=160, callbacks=[tb_callback]);

Epoch 1/160
5/5 [==============================] - 5s 42ms/step - loss: 2.0346 - categorical_accuracy: 0.1618
Epoch 2/160
5/5 [==============================] - 0s 40ms/step - loss: 1.8178 - categorical_accuracy: 0.2794
Epoch 3/160
5/5 [==============================] - 0s 41ms/step - loss: 1.6825 - categorical_accuracy: 0.2647
Epoch 4/160
5/5 [==============================] - 0s 39ms/step - loss: 1.4991 - categorical_accuracy: 0.3897
Epoch 5/160
5/5 [==============================] - 0s 41ms/step - loss: 1.3768 - categorical_accuracy: 0.4559
Epoch 6/160
5/5 [==============================] - 0s 52ms/step - loss: 1.2133 - categorical_accuracy: 0.4706
Epoch 7/160
5/5 [==============================] - 0s 45ms/step - loss: 1.2499 - categorical_accuracy: 0.4191
Epoch 8/160
5/5 [==============================] - 0s 45ms/step - loss: 1.3369 - categorical_accuracy: 0.3824
Epoch 9/160
5/5 [==============================] - 0s 44ms/step - loss: 1.1515 - categorical_accuracy: 0.5221
Epoch 10/1

5/5 [==============================] - 0s 42ms/step - loss: 0.2813 - categorical_accuracy: 0.8676
Epoch 75/160
5/5 [==============================] - 0s 42ms/step - loss: 0.2961 - categorical_accuracy: 0.8750
Epoch 76/160
5/5 [==============================] - 0s 40ms/step - loss: 0.2503 - categorical_accuracy: 0.9044
Epoch 77/160
5/5 [==============================] - 0s 39ms/step - loss: 0.2323 - categorical_accuracy: 0.9191
Epoch 78/160
5/5 [==============================] - 0s 40ms/step - loss: 0.2143 - categorical_accuracy: 0.9191
Epoch 79/160
5/5 [==============================] - 0s 39ms/step - loss: 0.2090 - categorical_accuracy: 0.9265
Epoch 80/160
5/5 [==============================] - 0s 39ms/step - loss: 0.2006 - categorical_accuracy: 0.9338
Epoch 81/160
5/5 [==============================] - 0s 41ms/step - loss: 0.1820 - categorical_accuracy: 0.9265
Epoch 82/160
5/5 [==============================] - 0s 40ms/step - loss: 0.1983 - categorical_accuracy: 0.9191
Epoch 83/160
5

5/5 [==============================] - 0s 46ms/step - loss: 0.0106 - categorical_accuracy: 1.0000
Epoch 148/160
5/5 [==============================] - 0s 41ms/step - loss: 0.0098 - categorical_accuracy: 1.0000
Epoch 149/160
5/5 [==============================] - 0s 40ms/step - loss: 0.0077 - categorical_accuracy: 1.0000
Epoch 150/160
5/5 [==============================] - 0s 42ms/step - loss: 0.0074 - categorical_accuracy: 1.0000
Epoch 151/160
5/5 [==============================] - 0s 40ms/step - loss: 0.0070 - categorical_accuracy: 1.0000
Epoch 152/160
5/5 [==============================] - 0s 40ms/step - loss: 0.0064 - categorical_accuracy: 1.0000
Epoch 153/160
5/5 [==============================] - 0s 40ms/step - loss: 0.0059 - categorical_accuracy: 1.0000
Epoch 154/160
5/5 [==============================] - 0s 40ms/step - loss: 0.0056 - categorical_accuracy: 1.0000
Epoch 155/160
5/5 [==============================] - 0s 41ms/step - loss: 0.0053 - categorical_accuracy: 1.0000
Epoch 

In [35]:
model.save('best.h5')

In [38]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mp_detect(frame, holistic)
        
        # Draw landmarks
        mp_draw(image, results)
        
        # 2. Prediction logic
        keypoints = mp_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            cache = np.unique(predictions[-10:])
            print(len(cache), cache[0] == np.argmax(res))
            if len(cache) < 2 and cache[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors)

        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('MediaPipe Holistic', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
I
1 True
my
2 True
my
2 True
my
2 True
my
2 True
my
2 True
my
2 True
name
3 False
name
3 False
name
3 False
name
2 False
name
2 False
name
2 False
name
2 False
name
2 False
name
2 False
name
1 True
friend
2 False
friend
2 False
friend
2 False
friend
2 False
friend
2 False
friend
2 False
friend
2 False
friend
2 False
friend
2 False
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
friend
1 True
sign
2 True
learning
3 True
learning
3 True
learning
3 True
learning
3 True
learning
